<a href="https://colab.research.google.com/github/ksetdekov/HSE_DS/blob/master/07%20NLP/kaggle%20hw/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip3 install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ksetdekov","key":"8c4abff92599eb402d83c7a75a6e0341"}'}

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle competitions download -c toxic-comments-classification-apdl-2021

  0% 0.00/23.6k [00:00<?, ?B/s]
100% 23.6k/23.6k [00:00<00:00, 20.8MB/s]
  0% 0.00/1.12M [00:00<?, ?B/s]
100% 1.12M/1.12M [00:00<00:00, 76.0MB/s]
  0% 0.00/402k [00:00<?, ?B/s]
100% 402k/402k [00:00<00:00, 56.1MB/s]


In [6]:
!ls

kaggle.json  sample_submission.csv  train_data.csv.zip
sample_data  test_data.csv.zip


In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [8]:
train = pd.read_csv('train_data.csv.zip', compression='zip')
test = pd.read_csv('test_data.csv.zip', compression='zip')

In [9]:
train.toxic.describe()

count    10809.000000
mean         0.333796
std          0.471590
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: toxic, dtype: float64

In [10]:
train.sample(5)

,comment,toxic
7458,Куда ты там скрины кидаешь шизик?\n,1.0
10808,Это не компенсация проезда к месту отдыха - он...,0.0
2578,"Да, я использую VPN. В таком случае, у клиента...",0.0
2485,Ну вот главврач и старается уменьшить вероятно...,0.0
2032,"Что-то размером с ВолгоБалт, может даже какая-...",0.0


In [11]:
test.sample(5)

,comment_id,comment
949,949,"По цвету - красиво, но непонятно, что хотел ск..."
3293,3293,оследние пару недель у него веселые стримы был...
3171,3171,"Хохлы-объебосы порвались, лол\n"
3133,3133,"В целом, согласен. Экономические и типа того (..."
2671,2671,"Эээ, не: Толян - порашный пацан.\n"


In [12]:
x_train, x_test, y_train, y_test = train_test_split(train.comment, train.toxic, random_state=0, stratify=train.toxic)

In [13]:
y_train.describe()

count    8106.000000
mean        0.333827
std         0.471608
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: toxic, dtype: float64

In [14]:
y_test.describe()

count    2703.000000
mean        0.333703
std         0.471622
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: toxic, dtype: float64

## Bag of words


In [15]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer
from nltk import ngrams

In [16]:
vec = CountVectorizer(ngram_range=(1, 2)) # строим BoW для слов
bow = vec.fit_transform(x_train) 

In [17]:
list(vec.vocabulary_.items())[:10]

[('ну', 103517),
 ('один', 107149),
 ('из', 57664),
 ('взял', 19194),
 ('карту', 65379),
 ('воспользовался', 23097),
 ('уже', 182088),
 ('профит', 140715),
 ('девочке', 36530),
 ('все', 24531)]

In [18]:
bow.mean()

0.0002405542888719116

In [19]:
clf = LogisticRegression(random_state=0, max_iter=500, class_weight='balanced')
clf.fit(bow, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=500, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

         0.0       0.90      0.87      0.89      1858
         1.0       0.74      0.79      0.76       845

    accuracy                           0.85      2703
   macro avg       0.82      0.83      0.82      2703
weighted avg       0.85      0.85      0.85      2703



In [21]:
test

,comment_id,comment
0,0,"В данном случае не пионервожатая, а техничка, ..."
1,1,В данном посте рассматривается обычный ленточн...
2,2,"Как я понимаю, у поверхностей отражаемость пог..."
3,3,Австралийские пауки съедят все живое на планете))
4,4,У нас раньше Амур пиво целые бутылки принимал ...
...,...,...
3598,3598,"Я всё жду, когда эта херня закончится\n"
3599,3599,Откуда такой акцент на Москве? У нас между про...
3600,3600,"Кого угодно, но не этих двух отбросов: свинью ..."
3601,3601,А как же радиоуправляемые машинки в клубах?\n


In [22]:
bow_test_pred = test.copy()
bow_test_pred['toxic'] = clf.predict(vec.transform(test.comment))
bow_test_pred['toxic'] = bow_test_pred['toxic'].astype(int)
bow_test_pred.drop('comment', axis=1, inplace=True)
bow_test_pred

,comment_id,toxic
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
3598,3598,1
3599,3599,0
3600,3600,1
3601,3601,0


In [23]:
bow_test_pred.to_csv('bow_v1.csv', index=False)

In [24]:
# !kaggle competitions submit -c toxic-comments-classification-apdl-2021 -f bow_v1.csv -m "kirill_setdekov first bow v1 submission"

## TF-IDF


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf2 = LogisticRegression(random_state=1, max_iter = 500)
clf2.fit(bow, y_train)
pred = clf2.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

         0.0       0.97      0.79      0.87      2214
         1.0       0.48      0.89      0.63       489

    accuracy                           0.81      2703
   macro avg       0.73      0.84      0.75      2703
weighted avg       0.88      0.81      0.83      2703



In [27]:
tf_idf = test.copy()
tf_idf['toxic'] = clf2.predict(vec.transform(test.comment))
tf_idf['toxic'] = tf_idf['toxic'].astype(int)
tf_idf.drop('comment', axis=1, inplace=True)
tf_idf

,comment_id,toxic
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
3598,3598,1
3599,3599,0
3600,3600,0
3601,3601,0


In [28]:
tf_idf.to_csv('tf_idf_v1.csv', index=False)

In [29]:
# !kaggle competitions submit -c toxic-comments-classification-apdl-2021 -f tf_idf_v1.csv -m "kirill_setdekov tfidf v1 submission"

## Symbol n-Grams

In [30]:
vec = CountVectorizer(analyzer='char', ngram_range=(1, 5))
bowsimb = vec.fit_transform(x_train)


In [31]:
from sklearn.preprocessing import MaxAbsScaler 
scaler = MaxAbsScaler()
scaler.fit(bowsimb)

MaxAbsScaler(copy=True)

In [32]:
bowsimb = scaler.transform(bowsimb)

In [33]:
clf3 = LogisticRegression(random_state=0,  max_iter=1000)
clf3.fit(bowsimb, y_train)
pred = clf3.predict(scaler.transform(vec.transform(x_test)))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

         0.0       0.97      0.87      0.91      2000
         1.0       0.71      0.91      0.80       703

    accuracy                           0.88      2703
   macro avg       0.84      0.89      0.86      2703
weighted avg       0.90      0.88      0.88      2703



In [34]:
importances = list(zip(vec.vocabulary_, clf.coef_[0]))
importances[0]

('н', 0.6167060139647299)

In [35]:
sorted_importances = sorted(importances, key = lambda x: -abs(x[1]))
sorted_importances[:20]

[('з. чт', 2.4054134367150373),
 ('ы сол', 2.3919589599163023),
 ('ьерн', 2.2785810515459537),
 (' горш', -1.979976614624312),
 ('мотья', 1.7435157860781292),
 ('май, ', 1.7058523876067757),
 ('этой ', 1.6475416855681653),
 ('гора', 1.4710723026209482),
 ('совр', 1.4531765870631195),
 ('их ,', 1.4233307071907748),
 ('я не ', 1.3989481978199367),
 ('т да ', 1.3887901729354284),
 ('ьни м', 1.3499673463315913),
 ('т 8', 1.315935088940947),
 ('м у с', 1.312410067186944),
 ('нубу', 1.2604893126358026),
 ('тушин', -1.2165199192811813),
 ('оркут', -1.215621304958302),
 ('дура ', 1.2140363947432982),
 ('унта ', 1.2002130316125421)]

In [36]:
symbol_ngrams = test.copy()
symbol_ngrams['toxic'] = clf3.predict(scaler.
                                      transform(vec.transform(test.comment)))
symbol_ngrams['toxic'] = tf_idf['toxic'].astype(int)
symbol_ngrams.drop('comment', axis=1, inplace=True)
symbol_ngrams

,comment_id,toxic
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
3598,3598,1
3599,3599,0
3600,3600,0
3601,3601,0


In [37]:
symbol_ngrams.to_csv('symbol_ngrams_v1.csv', index=False)

In [38]:
from sklearn.metrics import confusion_matrix

In [39]:
confusion_matrix(symbol_ngrams.toxic, tf_idf.toxic)

array([[2918,    0],
       [   0,  685]])

In [40]:
# !kaggle competitions submit -c toxic-comments-classification-apdl-2021 -f symbol_ngrams_v1.csv -m "kirill_setdekov symbol_ngrams_v1 v1 submission"

#FastText

In [41]:
!pip3 install fasttext
import fasttext

     |████████████████████████████████| 68 kB 2.9 MB/s 
  Using cached pybind11-2.8.0-py2.py3-none-any.whl (207 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3119042 sha256=374434ef7be700242ae71cae53500120da5af4bfcab359384f3cae2d294374c5
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [42]:
with open('ft_train_data.txt', 'w') as f:
    for pair in list(zip(x_train, y_train)):
        text, label = pair
        f.write(f'__label__{label} {text.lower()}\n')
        
with open('ft_test_data.txt', 'w') as f:
    for pair in list(zip(x_test, y_test)):
        text, label = pair
        f.write(f'__label__{label} {text.lower()}\n')

In [43]:
classifier = fasttext.train_supervised('ft_train_data.txt')#, 'model')
result = classifier.test('ft_test_data.txt')
print('P@1:', result[1])#.precision)
print('R@1:', result[2])#.recall)
print('Number of examples:', result[0])#.nexamples)

P@1: 0.8146503884572697
R@1: 0.8146503884572697
Number of examples: 2703


In [44]:
prediction = []
for item in [i.lower() for i in test.comment]:
   item = item.replace("\n"," ")
   prediction.append(classifier.predict(item)[0])

In [45]:
pred = [int(label[0].split('__')[2][0]) for label in prediction]


In [46]:
fasttext_pred = test.copy()
fasttext_pred['toxic'] = pred
fasttext_pred.drop('comment', axis=1, inplace=True)
fasttext_pred

,comment_id,toxic
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
3598,3598,1
3599,3599,0
3600,3600,1
3601,3601,0


In [47]:
confusion_matrix(symbol_ngrams.toxic, fasttext_pred.toxic)

array([[2652,  266],
       [  54,  631]])

In [48]:
fasttext_pred.to_csv('fasttext_pred_v1.csv', index=False)

In [49]:
# !kaggle competitions submit -c toxic-comments-classification-apdl-2021 -f fasttext_pred_v1.csv -m "kirill_setdekov fasttext_pred v1 submission"

## CNN

In [212]:
from torchtext.legacy import data

In [213]:
pd.read_csv('train_data.csv.zip', compression='zip')

,comment,toxic
0,Преступление и наказание\n,0.0
1,"И именно эти неработающие весы показывают, что...",0.0
2,"В Японии такие панельки, ебанько.\n",0.0
3,Еще у нас выявляют трещины с помощью белой кра...,0.0
4,"Дочитал до поезда в Норильск , дальше не стал\n",0.0
...,...,...
10804,А у мамы в группе до самого выпуска из сада та...,0.0
10805,Сука тупой дегенарт. Вот на эти видео. Съеби у...,1.0
10806,"В племенах украинцев, особенно западных, с дет...",1.0
10807,"Пост про жадность, о том, как человек оплативш...",0.0


In [214]:
!unzip train_data.csv.zip
!unzip test_data.csv.zip


Archive:  train_data.csv.zip
replace train_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train_data.csv          
Archive:  test_data.csv.zip
replace test_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test_data.csv           


In [215]:
# классы Field и LabelField отвечают за то, как данные будут храниться и обрабатываться при считывании
TEXT = data.Field(tokenize='spacy') # spacy -- значит, токенизацию будет делать модуль 
LABEL = data.LabelField()

ds = data.TabularDataset(
  path='train_data.csv', format='csv', 
  skip_header=True,
  fields=[('comment', TEXT),
        ('toxic', LABEL)]
)

In [293]:
pd.read_csv('test_data.csv')

,comment_id,comment
0,0,"В данном случае не пионервожатая, а техничка, ..."
1,1,В данном посте рассматривается обычный ленточн...
2,2,"Как я понимаю, у поверхностей отражаемость пог..."
3,3,Австралийские пауки съедят все живое на планете))
4,4,У нас раньше Амур пиво целые бутылки принимал ...
...,...,...
3598,3598,"Я всё жду, когда эта херня закончится\n"
3599,3599,Откуда такой акцент на Москве? У нас между про...
3600,3600,"Кого угодно, но не этих двух отбросов: свинью ..."
3601,3601,А как же радиоуправляемые машинки в клубах?\n


In [294]:
test = data.TabularDataset(
  path='test_data.csv', format='csv', 
  skip_header=True,
  fields=[('id', TEXT), ('comment', TEXT)]
)

In [217]:
next(ds.comment)

['Преступление', 'и', 'наказание']

In [218]:
next(ds.toxic)

'0.0'

In [219]:
TEXT.build_vocab(ds, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(ds)

In [220]:
TEXT.vocab.itos[:20]

['<unk>',
 '<pad>',
 ',',
 '.',
 'и',
 'в',
 'не',
 '-',
 'на',
 'что',
 '?',
 'с',
 'то',
 'это',
 'а',
 ')',
 'как',
 'по',
 'за',
 'у']

In [270]:
len(TEXT.vocab.itos)

25002

In [221]:
train, val = ds.split() # дефолтное соотношение 0.7


In [222]:
print(len(train))
print(len(val))
print(len(test))

7566
3243
3603


In [352]:
BATCH_SIZE  = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, val, test), 
    batch_size=BATCH_SIZE, 
    sort=True,
    sort_key=lambda x: len(x.comment), # сорируем тексты по длине, чтобы рядом оказывались предложения с одинаковой длиной и добавлялось меньше паддинга
    repeat=False)

In [353]:
for i, batch in enumerate(valid_iterator):
  print(batch.batch_size)
  # pass

64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
43


In [354]:
batch.fields

dict_keys(['comment', 'toxic'])

In [355]:
batch.batch_size

43

In [356]:
batch.comment

tensor([[24762,   156, 14164,  ...,  9454,   951,     0],
        [    2,  1105,     2,  ...,   983,     0,  6026],
        [   16,  3500,     9,  ...,     2,     7,     9],
        ...,
        [    7,     1,     1,  ...,     1,     1,     1],
        [10052,     1,     1,  ...,     1,     1,     1],
        [    3,     1,     1,  ...,     1,     1,     1]])

In [357]:
batch.toxic

tensor([1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0])

In [358]:
len(batch.toxic)

43

In [359]:
import torch.nn as nn

In [360]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout_proba):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[0], embedding_dim))
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[1], embedding_dim))
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[2], embedding_dim))
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout_proba)
        
    def forward(self, x):
        #x = [sent len, batch size]
        # print(x.shape)
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        embedded = self.embedding(x)
        #print(embedded.shape)
                
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        
        
        #embedded = [batch size, 1, sent len, emb dim]
        conv_0 = self.conv_0(embedded)
        #print(conv_0.shape)
        conv_0 = conv_0.squeeze(3)
        #print(conv_0.shape)
        conved_0 = F.relu(conv_0)
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        # print(conved_0.shape)
        pool_0 = F.max_pool1d(conved_0, conved_0.shape[2])
        # print(pool_0.shape)

        pooled_0 = pool_0.squeeze(2)
        # print(pooled_0.shape)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

In [361]:
import torch.nn.functional as F

def binary_accuracy(preds, y):
    rounded_preds = torch.round(F.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [362]:
def train_func(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        
        predictions = model(batch.comment.cuda()).squeeze(1)

        loss = criterion(predictions.float(), batch.toxic.float().cuda())
        acc = binary_accuracy(predictions.float(), batch.toxic.float().cuda())
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss
        epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [363]:
def evaluate_func(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.comment.cuda()).squeeze(1)

            loss = criterion(predictions.float(), batch.toxic.float().cuda())
            acc = binary_accuracy(predictions.float(), batch.toxic.float().cuda())

            epoch_loss += loss
            epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [364]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = 1
DROPOUT_PROBA = 0.5

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT_PROBA)

In [365]:
INPUT_DIM

25002

In [366]:
model

CNN(
  (embedding): Embedding(25002, 100)
  (conv_0): Conv2d(1, 100, kernel_size=(2, 100), stride=(1, 1))
  (conv_1): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
  (conv_2): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
  (fc): Linear(in_features=300, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [367]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1077,  0.1105,  0.5981,  ..., -0.8316,  0.4529,  0.0826],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [368]:
import torch.optim as optim

In [369]:
optimizer = optim.Adam(model.parameters()) # мы подали оптимизатору все параметры -- значит, эмбеддиги тоже будут дообучаться
criterion = nn.BCEWithLogitsLoss() # бинарная кросс-энтропия с логитами

model = model.cuda() # будем учить на gpu! =)

In [370]:
model.embedding

Embedding(25002, 100)

In [371]:
from torchsummary import summary

# summary(model, (14))


In [372]:
import torch

In [373]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_func(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_func(model, valid_iterator, criterion)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 01, Train Loss: 0.589, Train Acc: 67.89%, Val. Loss: 0.580, Val. Acc: 68.86%
Epoch: 02, Train Loss: 0.515, Train Acc: 74.34%, Val. Loss: 0.506, Val. Acc: 74.76%
Epoch: 03, Train Loss: 0.338, Train Acc: 86.19%, Val. Loss: 0.464, Val. Acc: 80.15%
Epoch: 04, Train Loss: 0.177, Train Acc: 93.33%, Val. Loss: 0.464, Val. Acc: 81.81%
Epoch: 05, Train Loss: 0.091, Train Acc: 96.75%, Val. Loss: 0.451, Val. Acc: 83.92%


In [374]:
test.examples

 ...]

In [375]:
model.eval()
cnn_res = []
with torch.no_grad():
    for batch in test_iterator:
        predictions = model(batch.comment.cuda())
        cnn_res.append(predictions)
        

In [376]:
testout = pd.read_csv('test_data.csv.zip', compression='zip')

In [377]:
cnnpred = testout.copy()
cnnpred['toxic'] = [float(item) for sublist in cnn_res for item in sublist]
cnnpred.drop('comment', axis=1, inplace=True)
cnnpred

,comment_id,toxic
0,0,-2.958906
1,1,4.002089
2,2,-0.402201
3,3,-2.432245
4,4,3.103574
...,...,...
3598,3598,-5.010084
3599,3599,1.808309
3600,3600,-0.488773
3601,3601,-0.287333


In [381]:
cnnpred['toxic'] = (cnnpred['toxic'] > 0.5).astype(int)
cnnpred

,comment_id,toxic
0,0,1
1,1,0
2,2,1
3,3,1
4,4,0
...,...,...
3598,3598,1
3599,3599,0
3600,3600,1
3601,3601,1


In [382]:
cnnpred.to_csv('cnnpred_v2.csv', index=False)

In [383]:
!kaggle competitions submit -c toxic-comments-classification-apdl-2021 -f cnnpred_v1.csv -m "kirill_setdekov cnn v2 submission"

100% 23.6k/23.6k [00:09<00:00, 2.64kB/s]
400 - Bad Request
